In [1]:
# The %... is an iPython thing, and is not part of the Python language.
# In this case we're just telling the plotting library to draw things on
# the notebook, instead of on a separate window.
%matplotlib inline
# See all the "as ..." contructs? They're just aliasing the package names.
# That way we can call methods like plt.plot() instead of matplotlib.pyplot.plot().
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import requests
import json

#PJM Has a REST API

PJM has documentation for a REST API available at http://www.pjm.com/~/media/etools/data-miner/user-guide.ashx

We'll use the API to get the data.

In [2]:
#These are all the nodes we're interested in so far.
nodeList = [
    5021673,
    32417525,
    32417527,
    32417545,
    32417547,
    32417599,
    32417601,
    32417629,
    32417631,
    32417633,
    32417635
]
#This is the base URL for the PJM REST API
url = 'https://dataminer.pjm.com/dataminer/rest/public/api/markets/realtime/lmp/daily'


In [3]:
def splitDateTime(utchour):
    #split datetime into date and time components
    datetime_parts = utchour.split('T', 1)
    parts = dict(date = datetime_parts[0], time = datetime_parts[1].rstrip('Z'))
    return parts

In [13]:
from datetime import datetime, timedelta, date, time

#Using code adapted from http://stackoverflow.com/questions/1060279/iterating-through-a-range-of-dates-in-python
def daterange(start, end):
    for n in range(int((end - start).days)):
        yield start + timedelta(n)
        
def formatDate(aDate):
    return aDate.strftime('%Y-%m-%d')

def adjustTime(parts):
    dtstring = parts['date'] + ' ' + parts['time']
    dtformat = '%Y-%m-%d %H:%M:%S'
    adjusted = datetime.strptime(dtstring, dtformat) - timedelta(hours = 4)
    return adjusted

def getHour(adjustedDatetime):
    t = adjustedDatetime.time()
    tstring = t.strftime('%H:%M:%S')
    tparts = tstring.split(':', 2)
    return tparts[0]

In [9]:
#HERE IS THE BIG KAHUNA
#This will take a long-ass time to run (25 mins), because we have to loop over every day in the years 2008 - 2012
from datetime import date
#set up our json POST data
params_list = [
    dict(startDate = formatDate(date(2008, 1, 1)), endDate = formatDate(date(2008, 12, 31)), pnodeList = nodeList),
    dict(startDate = formatDate(date(2009, 1, 1)), endDate = formatDate(date(2009, 12, 31)), pnodeList = nodeList),
    dict(startDate = formatDate(date(2010, 1, 1)), endDate = formatDate(date(2010, 12, 31)), pnodeList = nodeList),
    dict(startDate = formatDate(date(2011, 1, 1)), endDate = formatDate(date(2011, 12, 31)), pnodeList = nodeList),
    dict(startDate = formatDate(date(2012, 1, 1)), endDate = formatDate(date(2012, 12, 31)), pnodeList = nodeList),
    dict(startDate = formatDate(date(2013, 1, 1)), endDate = formatDate(date(2013, 12, 31)), pnodeList = nodeList),
    dict(startDate = formatDate(date(2014, 1, 1)), endDate = formatDate(date(2014, 12, 31)), pnodeList = nodeList),
    dict(startDate = formatDate(date(2015, 1, 1)), endDate = formatDate(date(2015, 11, 1)), pnodeList = nodeList)
]

results_dict = {}

for i in range(0, len(params_list)):
    
    #make the API call
    r = requests.post(url, json = params_list[i])
    if r.status_code == requests.codes.ok:
        results_dict[i] = r.json()
    else:
        r.raise_for_status()
        
    #be nice to the API, wait 2 seconds
    time.sleep(2)

In [14]:
recordsList = []
for result in results_dict.values():
    
    #make a new row for each individual price
    for record in result:
        #we are only interested in Total LMP per Sam's email
        if record['priceType'] == 'TotalLMP':
            data = {}
            data['pnodeId'] = record['pnodeId']
            published = splitDateTime(record['publishDate'])
            data['publishDate'] = published['date']
            for p in record['prices']:
                utcparts = splitDateTime(p['utchour'])
                hour = getHour(adjustTime(utcparts))
                if hour == '00':
                    hour = '24'
                key = 'price_' + hour
                data[key] = p['price']
            recordsList.append(data)

In [15]:
#let's see what we have. It's probably obscenely huge.
#print rawdf.shape
#results_dict[0][0:3]
print len(recordsList)
#recordsList[0:2]

31482


In [16]:
rawdf = pd.DataFrame(recordsList)

In [17]:
rawdf.to_csv('rawdf_pjm_realtime.csv')

In [18]:
rawdf.head()

,pnodeId,price_01,price_02,price_03,price_04,price_05,price_06,price_07,price_08,price_09,price_10,price_11,price_12,price_13,price_14,price_15,price_16,price_17,price_18,price_19,price_20,price_21,price_22,price_23,price_24,publishDate
0,32417601,2.48,14.23,13.66,21.55,21.44,21.79,22.94,24.15,23.06,26.53,29.16,26.03,27.36,26.25,26.41,26.57,28.61,65.92,72.45,75.68,76.83,46.26,44.68,28.91,2008-01-01
1,32417601,26.73,31.48,30.66,27.92,35.38,31.03,41.06,90.86,108.73,138.98,102.04,89.39,80.84,92.72,52.36,28.21,52.29,85.35,129.36,153.18,161.66,102.92,98.31,65.93,2008-01-02
2,32417601,191.99,60.30,45.30,70.30,61.71,49.43,56.30,113.23,105.20,78.29,45.28,52.23,69.05,66.20,53.44,31.91,55.90,31.72,73.64,68.42,76.62,70.43,57.13,63.90,2008-01-03
3,32417601,40.36,35.20,33.61,27.98,24.98,30.54,41.90,66.48,78.02,65.31,68.80,38.89,47.09,33.41,24.32,26.75,28.74,60.48,44.26,38.80,38.36,29.07,30.21,26.46,2008-01-04
4,32417601,35.13,34.19,25.23,24.42,23.09,24.00,23.54,30.27,37.55,56.69,67.66,56.65,33.43,38.33,29.52,33.23,39.43,69.88,45.50,42.07,35.96,35.54,24.52,27.46,2008-01-05


In [82]:
grouped = rawdf.groupby('publishDate')
for k, v in grouped[:3]:
    print k

KeyError: 'Column not found: 0'